<a href="https://colab.research.google.com/github/masayasato0407/pycaret_classification/blob/main/pycaret_classification_Randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pycaretはtrain sampleとtest sampleを予め分けて準備し、5つのステップでtest sampleでの精度を解析する

①training dataとtest dataを読み込む

②狭義のtraining set (training sample全体の7割)を10分割交差検証で比較しハイパーパラメーターのチューニングする

③validation set (training sample全体の3割)も使用して、training sample全体でのモデルを作成する

④training set全体での精度の検証 (internal validation)

⑤予め別に自身で準備したtest sampleで精度を検証する (external validation)
→この精度を上げるのが目的


In [ ]:
#パッケージのインストール
!pip install pycaret

In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd

In [ ]:
from pycaret.classification import *

#ここからは実際のデータの読み込み

In [ ]:
# ローカルファイルを直接アップロードする (train data)
from google.colab import files
uploaded = files.upload()


In [ ]:
#train dataの読み込み
train_data = pd.read_csv('train.csv')
train_data.head()

In [ ]:
# ローカルファイルを直接アップロードする (test data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#test dataの読み込み
test_data = pd.read_csv('test.csv')
test_data.head()

#ここからステップ①、pycaretにデータの型を判別させ、training data全体ををtrainingのための狭義のtraining sampleとパラメーターを決めるためのvalidation sampleに分割する。 


In [ ]:
#train_sizeを指定しない場合は7:3に分割される
#pycaretにデータの型を判別させる (sampleIDは無視する→ignore)
exp1 = setup(train_data, target = 'event', train_size = 0.8,ignore_features = ['sampleID'])

#ここからステップ② 狭義のtraining set (上でtraining sample全体の8割に指定)を10分割交差検証で比較しハイパーパラメーターのチューニングする

In [ ]:
#transformed train sampleでのmodelの比較 (デフォルトは10分割交差検証)
compare_models()

In [ ]:
#ランダムフォレストモデルを作成
rf = create_model('rf')

ここからハイパーパラメーターをチューニング、pycaretはランダムサーチで最適パラメーターを探索 

In [ ]:
#試行回数の指定 n_iter = 10 (時間がかかるので10にしています、50や100に設定してください)
#AUCがいいものを選ぶ場合はoptimizeを'AUC'で指定、正答率なら'Accuracy'

#ランダムフォレスト
tuned_rf = tune_model(rf,  n_iter = 10, fold=10,optimize = 'AUC')

#ここからステップ③ validation sampleも含めたtraining sample2100例全体でのモデル (final model)を固定する

In [ ]:
#ランダムフォレストモデルの固定
final_rf = finalize_model(tuned_rf)

#ここからステップ④　training set全体での精度検証 (internal validation)

このコードは厳密にはtraining sample全体で評価を行っているのではなく、training sampleの一部である (今回ならtraining sampleの2割)　validation sampleのみでの評価を行っている

→training sample全体で行うためには、本プロセスの最後で行っているtest dataでの評価の際に用いるtest dataをtraining dataに置き換えて、test dataに行うのと同様にScoreのCSVファイルを出力する

In [ ]:
#ランダムフォレストモデルのinternal validation
predict_model(final_rf);

#ここからステップ⑤　予め別に自身で準備したtest sampleで精度を検証する (external validation)→この精度を上げるのが目的

In [ ]:
#ランダムフォレストモデルの評価
prediction_rf = predict_model(final_rf, data = test_data)
prediction_rf

#既存のスコアリングシステムなどとの比較をしやすくするため、統計ソフトでの解析用に機械学習が出したスコアと正解ラベルをcsvファイルで出力する。

注意点: AUCを出すためにはprediction_label=0のものは、prediction=1の確率に変換をする

In [ ]:
#ランダムフォレストの予測出力 (左のフォルダにcsvが出ているので、ファイル右の点3つからダウンロード)
prediction_rf.to_csv('prediction_rf.csv', columns=['event','prediction_label','prediction_score'])

#ハイパーパラメーターや変数の重要度等を一斉に出力する(catboostなどでは出力されない)

In [ ]:
evaluate_model (final_rf)